In [1]:
import os
import keras
import numpy as np
import pandas as pd
import cv2
from keras.models import Model , Sequential
from keras.layers import Input, Conv3D , MaxPooling3D , Dropout, Activation, Flatten , Dense , BatchNormalization
import re

from keras.models import load_model

Using TensorFlow backend.


In [2]:
#load data

path = '/floyd/input/data'
data = []
target = []

for i in os.listdir(path):
    cap = cv2.VideoCapture(os.path.join(path,i))
    video = []
    label = int(re.sub('.mp4$','',i))
    target.append([label])
    while(cap.isOpened()):
        ret , frame = cap.read()
        if(ret == False):
            break
        frame = cv2.resize(frame , (60,60))
        frame = frame.astype(float)
        video.append(frame)
    cap.release()
    cv2.destroyAllWindows()
    video = np.array(video)
    data.append(video)
    
data = np.array(data)
target = np.array(target)



In [3]:
#lets take 60 frames from all frames with uniform distance
extract = []
for i in data:
    d = len(i)//60;
    video = []
    for j in range(0,len(i)):
        video.append(i[j])
        j+=d;
        if(len(video)==60):
            break
    extract.append(video)

data = np.array(extract)

In [4]:
data.shape

(66, 60, 60, 60, 3)

In [5]:
target = keras.utils.to_categorical(target)

In [6]:
target.shape

(66, 66)

In [35]:
'''define model'''
in_shape = (60 , 60 , 60 , 3)
model = Sequential()
model.add(Conv3D(6, (4, 4 , 4), padding='same', input_shape=in_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv3D(6,(4, 4 , 4)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size = (2,2,2)))


model.add(Conv3D(12 , (3, 3 , 3) , padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv3D(6,(4,4 , 4)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size = (3,3 , 3)))
model.add(Flatten())
model.add(Dense(3072))
model.add(Activation('relu'))
model.add(Dense(66))
model.add(Activation('softmax'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss = 'categorical_crossentropy' , optimizer = opt , metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 60, 60, 60, 6)     1158      
_________________________________________________________________
batch_normalization_29 (Batc (None, 60, 60, 60, 6)     24        
_________________________________________________________________
activation_43 (Activation)   (None, 60, 60, 60, 6)     0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 57, 57, 57, 6)     2310      
_________________________________________________________________
batch_normalization_30 (Batc (None, 57, 57, 57, 6)     24        
_________________________________________________________________
activation_44 (Activation)   (None, 57, 57, 57, 6)     0         
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 28, 28, 28, 6)     0         
__________

In [36]:
model.fit(data , target , epochs = 60)

Epoch 1/60
66/66 [==============================] - 5s 74ms/step - loss: 6.4761 - acc: 0.0000e+00
Epoch 2/60
66/66 [==============================] - 3s 40ms/step - loss: 13.1863 - acc: 0.0152
Epoch 3/60
66/66 [==============================] - 3s 40ms/step - loss: 7.0587 - acc: 0.0455
Epoch 4/60
66/66 [==============================] - 3s 40ms/step - loss: 6.2701 - acc: 0.0606
Epoch 5/60
66/66 [==============================] - 3s 40ms/step - loss: 5.9724 - acc: 0.0455
Epoch 6/60
66/66 [==============================] - 3s 40ms/step - loss: 5.3249 - acc: 0.1515
Epoch 7/60
66/66 [==============================] - 3s 40ms/step - loss: 5.1454 - acc: 0.0455
Epoch 8/60
66/66 [==============================] - 3s 40ms/step - loss: 4.8390 - acc: 0.1667
Epoch 9/60
66/66 [==============================] - 3s 40ms/step - loss: 4.3560 - acc: 0.0909
Epoch 10/60
66/66 [==============================] - 3s 40ms/step - loss: 4.2389 - acc: 0.1364
Epoch 11/60
66/66 [==============================] - 3

In [40]:
pred  = model.predict(data)
cnt= 0
for i in range(len(pred)):
    if(target[i].argmax() == pred[i].argmax()):
        cnt+=1
cnt

64

In [41]:
model.save('my_model.h5')